In [17]:
import requests
from bs4 import BeautifulSoup
#739

In [19]:
data = []
urls = []
for i in range(1,51):   
    base_url = f'https://iranfile.ir/properties/buy-apartment?page={i}'
    r = requests.get(base_url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('table', {'class':'table table-hover'})
    a_tags = table.find_all('a')
    for url in a_tags:
        urls.append(url.get('href'))
    urls = list(set(urls))



In [23]:
import sqlite3


def get_page_data(id, url):
    conn = sqlite3.connect('iranfile1.csv')
    cursor = conn.cursor()


    r = requests.get(url)
    r.status_code
    page_soup = BeautifulSoup(r.content, 'html.parser')
    table = page_soup.find('table', {'class':'table file-table file hidden-xxs hidden-xs for-print'})
    data = {}


    for label_tag in page_soup.find_all(class_='label-data'):
        label = label_tag.get_text(strip=True).replace(':', '')
        value_tag = label_tag.find_next(class_='file-data')

        if value_tag:
            value = value_tag.get_text(strip=True)
        else:
            container = label_tag.find_next(class_='full-address-popup-container')
            if container:
                value = container.get_text(strip=True)
            else:
                value = None 

        data[label] = value

    if 'تلفن' in data and data['تلفن'] in (None, '*******', ''):
        data['تلفن'] = 'مخفی'
    if 'نام مالک' in data and data['نام مالک'] in (None, '*******', ''):
        data['نام مالک'] = 'مخفی'
    if 'تلفن' in data:
        data['شماره تلفن'] = data.pop('تلفن')



    for row in page_soup.find_all('tr'):
        cells = row.find_all('td')
        if len(cells) == 2:
            key = cells[0].get_text(strip=True)
            value_td = cells[1]
            value = value_td.get_text(strip=True)
            checked_icon = value_td.find('i')
            if checked_icon:
                if 'file-checked' in checked_icon.get('class', []):
                    value = 'دارد'
                else:
                    value = 'ندارد'
        data[key] = value




    features = ''
    for item in page_soup.find_all(class_='check-box-info'):
        name = item.get_text(strip=True)
        if name:
            features += name + ' '
    data['تجهیزات و امکانات'] = features


    d = 0
    for item in page_soup.find_all(class_='info-melk-file-details'):
        d += 1
        if d == 5:
            break
        key_tag = item.find('div', class_=False) 
        value_tag = item.find(class_='file-data')
        if key_tag and value_tag:
            key = key_tag.get_text(strip=True)
            value = value_tag.get_text(strip=True)
            data[key] = value



    for item in page_soup.find_all(class_='col-xs-3'):
        name_tag = item.find('div')
        icon_tag = item.find('i')
        if name_tag:
            name = name_tag.get_text(strip=True)
            active = 'file-checked' in icon_tag.get('class', []) if icon_tag else False
            if active:
                data['موقعیت مکانی'] = name



    creat_table_statement = f'CREATE TABLE IF NOT EXISTS headline(id,'
    data_lst = list(data.keys())
    for item in data_lst:
        creat_table_statement += f'[{item}] TEXT,'
    creat_table_statement = creat_table_statement.rstrip(',') + ')'
    cursor.execute(creat_table_statement)
    columns = ['تاریخ', 'کد ملک', 'نوع ملک', 'نام مالک',
                'تلفن ', 'آدرس', 'تعداد طبقات', 'تعداد واحد در طبقه',
                  'قیمت کل', 'قیمت واحد', 'آنالیز قیمت', 'طبقه',
                    'زیربنا', 'خواب', 'تلفن', 'آشپزخانه', 'سرویس', 
                    'کفپوش', 'OPEN', 'پارکینگ', 'انباری', 'بالکن', 
                    'تجهیزات و امکانات', 'سن بنا', 'وضعیت ملک', 'نما',
                      'وضعیت سند', 'موقعیت مکانی']
    clean_data = {k.strip(): v for k, v in data.items()}
    values = [id] + [clean_data.get(col, None) for col in columns]

    placeholders = ','.join(['?'] * len(values))
    insert_sql = f'INSERT INTO headline VALUES ({placeholders})'
    cursor.execute(insert_sql, values)
    conn.commit()
    conn.close()




In [24]:
for id, url in enumerate(urls):
    try:
        get_page_data(id, url)
    except AttributeError as e:
        print(f'ERROR on {url}: {e}')
        continue



In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("iranfile1.csv")  # یا iranfile.db اگر اسم رو تغییر دادی

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("جدول‌ها:", tables)

df = pd.read_sql_query("SELECT * FROM headline;", conn)

print(df.head())

df.to_csv("iranfile_exported1.csv", index=False, encoding='utf-8-sig')

جدول‌ها:        name
0  headline
   id       تاریخ    کد ملک   نوع ملک   نام مالک تلفن   \
0   0  1404/07/12  10574688  آپارتمان       مخفی  None   
1   1  1404/07/13  10051662  آپارتمان  سلطان پور  None   
2   2  1404/07/13  10566251  آپارتمان       مخفی  None   
3   3  1404/07/14  10488297  آپارتمان       مخفی  None   
4   4  1404/07/14  10428640  آپارتمان       مخفی  None   

                                                آدرس تعداد طبقات  \
0                                    تهرانپارس خ 110           4   
1  بهارستان خ صفی علی شاه كوچه صفی بن بست علیشاه ...           4   
2                                     جشنواره خ زهدی           5   
3                                        صادقیه خ 15           5   
4                                  وحیدیه خ بخت آزاد           5   

  تعداد واحد در طبقه               قیمت کل  ...  OPEN پارکینگ انباری بالکن  \
0                  2   6,970,000,000 تومان  ...  دارد   ندارد   دارد  دارد   
1                  1   4,500,000,000 تومان  ...  دارد